Import Necessary libraries

In [ ]:
!pip install opendatasets
!pip install pandas
!pip install hazm
import opendatasets as od
import pandas
import pandas as pds
import hazm
import pandas as pd
from bs4 import BeautifulSoup
import re
from hazm import Normalizer
from hazm import word_tokenize
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
!pip install sentencepiece
!pip install accelerate -U
!pip install torch transformers datasets
import torch

from transformers import get_scheduler, EarlyStoppingCallback

from google.colab import drive

from datasets import load_dataset
from transformers import  AutoModelForSeq2SeqLM, MT5Tokenizer
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer
drive.mount('/content/drive')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
od.download(
    "https://www.kaggle.com/datasets/amirpourmand/tasnimdataset")


file =('tasnimdataset/tasnim.csv')
newData = pds.read_csv(file)


normalizer=Normalizer()
file =('tasnimdataset/tasnim.csv')
data = pd.read_csv(file)
data = data[['abstract', 'body']]
data = data.dropna()
data['abstract'] = data['abstract'].astype(str)
data['body'] = data['body'].astype(str)



tqdm.pandas()

data['normal_abstract'] = data['abstract'].progress_apply(normalizer.normalize)
data['normal_body'] = data['body'].progress_apply(normalizer.normalize)
data = data[data['normal_body'] != "منبع: انتهای پیام /"]
data.to_csv('cleaned_data.csv', index=False)
file =('cleaned_data.csv')
data = pd.read_csv(file)
data = data[['normal_abstract', 'normal_body']]
data.to_csv('final_dataset.csv', index=False)
data.to_csv('/content/drive/MyDrive/my_dataset.csv', index=False)


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: pardiszahraei
Your Kaggle Key: ··········


100%|██████████| 84.7M/84.7M [00:03<00:00, 24.8MB/s]


100%|██████████| 63494/63494 [05:39<00:00, 186.78it/s]


In [ ]:
np.random.seed(0)
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/my_dataset.csv')

train_val_test = dataset['train'].train_test_split(test_size=0.2)  # 80% for training, 20% for validation+test
val_test = train_val_test['test'].train_test_split(test_size=0.5)  # Split the 20% equally into validation and test

train_dataset = train_val_test['train']
val_dataset = val_test['train']
test_dataset = val_test['test']

#test_df = pd.DataFrame(test_dataset)

#test_df.to_csv('/content/drive/MyDrive/test_dataset.csv', index=False)

Replace 'your_token' with your actual token

In [ ]:
import os
os.environ['HF_TOKEN'] = 'your_token'


For this Task, We can use many models but what I chose in the end is **pn-summary-mt5-small** although its a small model and there are many larger models with far better performances, this model is small and the outputs are acceptable.For better results we can use the base model.

This code is for training a sequence-to-sequence language model using the Hugging Face’s Transformers library. Here’s a brief description of what each part does:

Model Loading: The model and tokenizer are loaded from a pretrained model, specifically ‘HooshvareLab/pn-summary-mt5-small’. Gradient checkpointing is enabled on the model to save memory during training.

Preprocessing Function: A preprocessing function is defined to convert the input data into a format suitable for the model. It tokenizes the inputs and labels, truncates them to a specified max length, and pads them to the max length.

Dataset Preparation: The preprocessing function is applied to the train, validation, and test datasets using the map function. This converts the datasets into the required format for training.

Training Arguments: Various training arguments are set using the TrainingArguments class. These include the output directory, number of epochs, batch sizes, learning rate, scheduler type, evaluation strategy, and more.

Trainer Initialization: A Trainer object is initialized with the model, training arguments, training dataset, and evaluation dataset. An EarlyStoppingCallback is added to stop training early if the validation loss does not improve for a certain number of evaluation steps.

Model Training: Finally, the model is trained using the train method of the Trainer class. The trained model will be saved in the directory specified in the training arguments.

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('HooshvareLab/pn-summary-mt5-small')
model.config.gradient_checkpointing = True

tokenizer = MT5Tokenizer.from_pretrained("HooshvareLab/pn-summary-mt5-small")

def preprocess_function(examples):
    inputs = [str(doc) for doc in examples['normal_body']]
    abstracts = [str(abs) for abs in examples['normal_abstract']]

    model_inputs = tokenizer(inputs, truncation=True, max_length=256, padding='max_length')

    with tokenizer.as_target_tokenizer():
      labels = tokenizer(abstracts, truncation=True, max_length=64, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=18,
    per_device_eval_batch_size=32,
    eval_accumulation_steps=1,
    prediction_loss_only=True,
    learning_rate=0.0001,
    lr_scheduler_type='linear',
    evaluation_strategy='steps',
    save_steps=500,
    save_total_limit=2,
    remove_unused_columns=True,
    run_name='run_name',
    gradient_accumulation_steps=4,
    logging_steps=500,
    eval_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()






config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/329 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/6349 [00:00<?, ? examples/s]

Map:   0%|          | 0/6350 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
500,2.089700,1.505315
1000,1.631300,1.471619
1500,1.579400,1.457491
2000,1.567000,1.444375
2500,1.548900,1.440721
3000,1.532300,1.437254
3500,1.536500,1.435199


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=3525, training_loss=1.6398141998940325, metrics={'train_runtime': 10144.0909, 'train_samples_per_second': 25.037, 'train_steps_per_second': 0.347, 'total_flos': 6.709727827329024e+16, 'train_loss': 1.6398141998940325, 'epoch': 5.0})

**Note**
<font color='red'>
In the Hugging Face’s Trainer API, the loss function does not need to be explicitly defined. The model I am using, AutoModelForSeq2SeqLM, is a sequence-to-sequence model which already has a built-in loss function suitable for sequence-to-sequence tasks. The Trainer class computes the loss during training. When we pass input_ids and labels to your model, it returns the loss in its outputs. This loss is then used by the Trainer for the backward pass and the optimization step.

</font><font color='red'>
The Bleu score is a metric for evaluating the quality of text which has been machine-translated from one natural language to another, it’s not a loss function. I have used it during evaluation to measure the performance of your model, but not for training.

</font><font color='red'>
The metric_for_best_model argument in TrainingArguments is used to specify which metric will be used to compare the models during evaluation. In my case, it’s set to 'loss', which means the model with the lowest loss will be considered the best and will be loaded at the end of training.

So, I don’t need to define a loss function in this case.</font>

In [ ]:
model_path = "/content/drive/MyDrive/my_finetuned_model"
tokenizer_path = "/content/drive/MyDrive/my_finetuned_tokenizer"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)


('/content/drive/MyDrive/my_finetuned_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/my_finetuned_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/my_finetuned_tokenizer/spiece.model',
 '/content/drive/MyDrive/my_finetuned_tokenizer/added_tokens.json')

In [ ]:
from datasets import DatasetDict

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

dataset_dict.save_to_disk("/content/drive/MyDrive/my_datasets")


Saving the dataset (0/1 shards):   0%|          | 0/50795 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6349 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6350 [00:00<?, ? examples/s]

Time to Evaulate!

In [ ]:
!pip install nltk bert_score rouge_score

In [ ]:
from transformers import AutoModelForSeq2SeqLM, MT5Tokenizer
from datasets import DatasetDict

dataset_dict = DatasetDict.load_from_disk("/content/drive/MyDrive/my_datasets")

model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/my_finetuned_model")
tokenizer = MT5Tokenizer.from_pretrained("/content/drive/MyDrive/my_finetuned_tokenizer")

train_dataset = dataset_dict['train']
val_dataset = dataset_dict['validation']
test_dataset = dataset_dict['test']


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Testing the Tokenizers decode and encode methods:  

In [ ]:
test_sentence = "یکی از رهبران معارض بحرین گفت: آل خلیفه با امکاناتی که در اختیار صهیونیست ها قرار داده وارد مرحله بسیار خطرناکی شده است که پیامد های آن گریبان تمام منطقه را خواهد گرفت."

input_ids = tokenizer.encode(test_sentence, return_tensors='pt')

print("Input IDs:", input_ids)

decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)

print("Decoded text:", decoded_text)


Input IDs: tensor([[  259,  7371,   695, 22819, 86058,   548, 53843, 40851,  2154,  5021,
           267,  8442,  7971, 17079,   376,   768,   858, 18237, 13499,   934,
           509,   858, 39398,   259, 42584, 92034,  2408,  1875,  5445,   259,
         14048,   259, 11373,   259, 21778,  2869,   614, 16634,   259, 29111,
         36571,   406,  2801,   950,   934,  3676, 60859,  1091,  1512, 76849,
         23597,   259,  6513,   259,  8179,   376,   916,  6590,   883,  8654,
           260,     1]])
Decoded text: یکی از رهبران معارض بحرین گفت: آل خلیفه با امکاناتی که در اختیار صهیونیست ها قرار داده وارد مرحله بسیار خطرناکی شده است که پیامد های آن گریبان تمام منطقه را خواهد گرفت.


Testing the models predictions:

In [ ]:
text_to_summarize = """
احسان ارکانی نماینده مردم نیشابور در مجلس شورای اسلامی در گفت‌وگو با خبرنگار پارلمانی  ، از ارائه طرح 2 فوریتی شفافیت قوای سه‌گانه دستگاه‌های اجرایی و نهادهای عمومی غیردولتی به هیئت رئیسه مجلس خبر داد. نماینده نیشابور در مجلس با بیان اینکه بیش از 200 تن از نمایندگان طرح شفافیت قوای سه‌گانه را امضا کرده‌اند،گفت: با توجه به اینکه، این طرح  با 2 فوریت تقدیم هیئت رئیسه شد طبق آیین‌نامه داخلی مجلس باید هفته آینده در دستورکار مجلس قرار گیرد. متن کامل طرح 2 فوریتی شفافیت قوای سه‌گانه دستگاه‌های اجرایی و نهادهای عمومی غیردولتی به شرح ذیل است: مقدمه (دلایل توجیهی): نظر به اینکه نظام مقدس جمهوری اسلامی ایران به تعبیر امامین انقلاب اسلامی متکی بر آرای عمومی است و لازمه این موضوع اعتماد عموم مردم به نمایندگان و مدیران و مسئولان اعم از انتخابی و انتصابی می‌باشد، برای ارتقای سرمایه اجتماعی در جمهوری اسلامی ایران و تقویت باور عمومی که هیچ چیز از مردم پنهان نبوده و تمامی مسئولان و دولتمردان و مدیران و نمایندگان در جمهوری اسلامی ایران به این امر باور دارند که باید اطلاعات مربوط به حوزه فعالیت دستگاه‌های متبوع خود را به همراه مذاکرات و تصمیمات متخذه که قاعدتا بر حقوق تمامی مردم یا بخش عمده‌ای از جامعه اثرگذار است در اختیار عموم مردم قرار دهند و امکان داوری را برای مردم فراهم سازند و اینکه تاکنون تلاش‌های صورت گرفته در این راستا نتوانسته موفقت‌آمیز بوده و بعضا به دلایل مختلف منجر به مخالفت‌هایی شده و بر همین اساس هم به نتیجه منتهی نشده است لذا (طرح شفافیت قوای سه‌گانه و دستگاه‌های اجرایی و سایر نهادها) به منظور رفع خلاء قانونی و تحولی در نظام حکمرانی کشور تهیه و به شرح زیر برای امضای نمایندگان محترم و تصویب در صحن علنی مجلس تقدیم می‌گردد. عنوان طرح: طرح شفافیت قوای سه گانه و دستگاه‌های اجرایی و سایر نهادها موضوع طرح: الف) دامنه شمول قانون ماده 1- دستگاه‌ها و نهادها و سازمان ها و موسسات مشمول این قانون عبارتند از: 1-1- کلیه دستگاه‌های اجرایی موضوع آمده 5 قانون مدیریت خدمات کشوری؛ از جمله هیئت وزیران و کلیه کمیسیون‌ها و دستگاه‌های متشکله مرکب از وزراء یا مدیران دستگاه‌های دولتی و کلیه وزارتخانه‌ها، سازمان‌ها، موسسات، شرکت‌های دولتی، موسسات انتفاعی وابسته به دولت، بانک‌ها و موسسات اعتباری دولتی، و همچنین دستگاه‌هایی که شمول قانون بر آنها مستلزم  ذکر یا تصریح نام است، اعم از اینکه قانون خاص خود را داشته و یا از قوانین و مقررات عام تبعیت نمایند، از جمله شرکت ملی نفت ایران، شرکت ملی گاز ایران، شرکت ملی صنایع پتروشیمی ایران، سازمان گسترش و نوسازی صنایع ایران، سازمان بنادر و کشتیرانی جمهوری اسلامی ایران، سازمان صدا و سیمای جمهوری اسلامی ایران و تمام موسسات و شرکت‌های تابع یا وابسته به آنها، دانشگاه‌ها و موسسات آموزشی و پژوهشی و موسسات و نهادهای عمومی غیر دولتی موضوع ماده 3 قانون مدیریت خدمات کشوری - مصوب 1386 - از جمله شهرداری‌ها، کمیته امداد امام خمینی، هلال احمر جمهوری اسلامی ایران و تمامی سازمان‌ها، موسسات و شرکت‌های تابع یا وابسته به آنها و شوراهای اسلامی شهر و روستا، بخش، شهرستان، استان و شورای عالی استانها. 1-2- قوه مقننه شامل مجلس شورای اسلامی، دیوان محاسبات کشور و تمامی سازمان‌ها، موسسات و شرکت‌های تابع یا وابسته به آنها. 1-3- قوه قضائیه شامل دادسراها و دادگاه‌های دادگستری اعم از دادگاه‌های عمومی و ویژه، سازمان بازرسی کل کشور، دیوان عدالت اداری، سازمان قضایی نیروهای مسلح و تمامی سازمان‌ها، موسسات و شرکت‌های تابع یا وابسته به آنها و همچنین شورای عالی حل اختلاف. 1-4- مجمع تشخیص مصلحت نظام، شورای عالی انقلاب فرهنگی، شورای عالی فضای مجازی و کلیه شوراهای عالی که به موجب قانون مصوب مجلس تشکیل شده‌اند. 1-5- کلیه موسسات عهده‌دار خدمات عمومی از جمله شامل کانون‌های وکلای دادگستری، کانون کارشناسان رسمی دادگستری، سازمان نظام پزشکی جمهوری اسلامی ایران، سازمان‌های نظام مهندسی، اتاق‌های بازرگانی، صنایع، معادن و کشاورزی ایران، اصناف و تعاون، دانشگاه آزاد اسلامی، دانشگاه‌ها و موسسات آموزش عالی، مدارس و موسسات اعتباری و شرکت‌های بیمه غیر دولتی. ب) نحوه اقدام ماده 2- مشمولین این قانون موظفند داده ها و اطلاعات عمومی و تصمیمات متخذه سازمان یا شورای متبوع خود را به موجب قوانین و مقررات از طریق سامانه‌ةای مربوط به خود منتشر و اطلاعات رسانی نمایند، به طوری که عدم بارگزاری هر کدام از اطلاعات، به معنای محرمانه بودن اطلاعات مربوط به حساب آید. ماده 3- رئیس مجلس شورای اسلامی و رئیس قوه قضائیه موظفند مصادیق داده و اطلاعات عمومی در هر کدام از نهادها و دستگاه‌های متبوع خود را علاوه بر مواردی که به عنوان اطلاعات عمومی در قانون انتشار و دسترسی آزاد به اطلاعات مشخص شده است معین کرده و پس از 4 ماه از تصویب این قانون جهت دسترسی عمومی به این اطلاعات به کمیسیون انتشار و دسترسی آزاد به اطلاعات ابلاغ نمایند. ماده 5: تمامی نهدهای شورایی کشور مشتمل بر مجلس شورای اسلامی، مجمع تشخیص مصلحت نظام، شوراهای شهر و روستاف شوراهای عالی کشور موظفند مشروح مذاکرات خود را اعم از صحن و کمیسیون‌های تابع آنها و آرای ماخوذه از اعضا را بلافاصله در پایگاه اطلاع‌رسانی خود منتشر نمایند. آیین‌نامه محرمانگی و غیرعلنی بودن جلسات هر کدام از نهادها و شوراها پس از سه ماه از تصویب این قانون باید تعیین و پس از تصویب شورای امنیت ملی به اطلاع عموم رسانده شود. ماده 6: وزارت ارتباطات و فناوری اطلاعات موظف است ظرف سه ماه از تاریخ لازم‌الاجرا شدن این قانون، با همکاری سازمان اداری و استخدامی کشور و دستگاه‌های ذیربط، ضوابط ایجاد، نگهداری و اشتراک‌گذاری اطلاعات شامل طراحی فرم‌ها و قالب‌های مورد قبول اسناد را برای تمامی دستگاه‌ها و نهادهای مشمول این قانون، به گونه‌ای تدوین کند و پس از تصویب شورای اجرای فناوری اطلاعات با تایید شورای امنیت ملی، جهت اجرا ابلاغ نماید که اطلاعات منتشر یا به اشتراک گذاشته شده توسط موسسات مشمول بدون وابستگی به فناوری‌های خاص توسط نرم‌افزارهای متنوع قابل استفاده و پردازش باشند. همچنین، ضوابط ایجاد، اداره و به‌روزرسانی پایگاه های اطلاع‌رسانی موسسات مشمول را از جهت محتوا، سهولت و نحوه دسترسی کاربران، قابل اعتماد بودن و قابل فهم بودن برای عموم مردم، توسط وزارت ارتباطات و فناوری اطلاعات تهیه شده، پس از تصویب شورای اجرایی فناوری اطلاعات جهت اجرا ابلاغ می‌گردد. ماده 7: عدم اجرا یا اجرای ناقص تکالیف مقرر در این قانون با انتشار اطلاعات مغایر مفاد این قانون تخلف محسوب و متخلف یا متخلفان حسب مورد بر اساس رای هیئت‌های رسیدگی به تخلفات اداری و با توجه به اهمیت موضوع به یکی از مجازات‌های اداری مقرر در بندهای (ج) تا (ی) ماده (9) قانون رسیدگی به تخلفات اداری - مصوب 1372 - یا مجازات متناسب مطابق قوانین مربوط محکوم می‌شوند. انتهای پیام/
"""
inputs = tokenizer(text_to_summarize, truncation=True, max_length=256, padding='max_length', return_tensors="pt")

summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=64, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)


نماینده نیشابور در مجلس از ارائه طرح ۲ فوریتی شفافیت قوای سه گانه دستگاه های اجرایی و نهادهای عمومی غیردولتی به هیئت رئیسه مجلس خبر داد.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLP_HW4_test_dataset.csv')
df

,normal_abstract,normal_body
0,تجارت ایران و روسیه در ۹ ماه سال ۱۴۰۰ از سه و ...,به گزارش، سید روح اله لطیفی، سخنگوی گمرک در آس...
1,در پی اظهارات محسن هاشمی رئیس شورای شهر تهران ...,به گزارش خبرنگار اجتماعی؛ محسن هاشمی رئیس شورا...
2,یک کارشناس آینده‌پژوهی با اشاره به بحران کاهش ...,به گزارش خبرنگار اجتماعی باشگاه خبرنگاران پویا...
3,رئیس‌جمهور در مراسم تشییع باشکوه و پرشور پیکر ...,به گزارش حوزه دولت، آیت‌الله سید ابراهیم رئیسی...
4,گروه استان‌ها- اداره میراث فرهنگی و گردشگری شه...,صیاد جعفری فرد در گفت‌وگو با خبرنگار تسنیم در ...
...,...,...
6132,رئیس شورای حاکمیتی سودان عادی‌سازی روابط با رژ...,به گزارش گروه بین‌الملل، پائیز سال گذشته بود ک...
6133,در روزهای اخیر، رئیس‌جمهور از بخش خصوصی درخواس...,به گزارش خبرنگار بهداشت و درمان؛ پیش از ایام ن...
6134,با پیروزی انقلاب اغلب هنرها از جمله موسیقی رشد...,، سعید تشکری: ادبیات انقلاب اسلامی و سقوط استب...
6135,مسعود فراستی بار دیگر در برنامه هفت به انتقاد ...,به گزارش خبرنگار فرهنگی، مسعود فراستی در میز ن...


In [ ]:
import pandas as pd
from tqdm import tqdm
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

df = pd.read_csv('/content/drive/MyDrive/NLP_HW4_test_dataset.csv')

predicted_abstracts = []

for text in tqdm(df['normal_body'], desc="Generating summaries"):
    inputs = tokenizer(text, truncation=True, max_length=256, padding='max_length', return_tensors="pt")

    inputs = inputs.to(device)

    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=64, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    predicted_abstracts.append(summary)

df['predicted_abstract'] = predicted_abstracts

# Save the DataFrame to a new CSV file
df.to_csv('/content/drive/MyDrive/final_test_dataset_with_predictions.csv', index=False)


Generating summaries: 100%|██████████| 6137/6137 [1:37:46<00:00,  1.05it/s]


In [ ]:
df

,normal_abstract,normal_body,predicted_abstract
0,تجارت ایران و روسیه در ۹ ماه سال ۱۴۰۰ از سه و ...,به گزارش، سید روح اله لطیفی، سخنگوی گمرک در آس...,سخنگوی گمرک گفت: از تجارت سه میلیون و ۵۵۸ هزار...
1,در پی اظهارات محسن هاشمی رئیس شورای شهر تهران ...,به گزارش خبرنگار اجتماعی؛ محسن هاشمی رئیس شورا...,رئیس شورای شهر تهران در دویست و هشتاد و نهمین ...
2,یک کارشناس آینده‌پژوهی با اشاره به بحران کاهش ...,به گزارش خبرنگار اجتماعی باشگاه خبرنگاران پویا...,رئیس پژوهشگاه علوم تحقیقات و فناوری اطلاعات گف...
3,رئیس‌جمهور در مراسم تشییع باشکوه و پرشور پیکر ...,به گزارش حوزه دولت، آیت‌الله سید ابراهیم رئیسی...,رئیس جمهور از تقاطع خیابان فلسطین به جمع تشییع...
4,گروه استان‌ها- اداره میراث فرهنگی و گردشگری شه...,صیاد جعفری فرد در گفت‌وگو با خبرنگار تسنیم در ...,گروه استان ها- رئیس اداره میراث فرهنگی و گردشگ...
...,...,...,...
6132,رئیس شورای حاکمیتی سودان عادی‌سازی روابط با رژ...,به گزارش گروه بین‌الملل، پائیز سال گذشته بود ک...,سودان با میانجیگری آمریکا توافقنامه عادی سازی ...
6133,در روزهای اخیر، رئیس‌جمهور از بخش خصوصی درخواس...,به گزارش خبرنگار بهداشت و درمان؛ پیش از ایام ن...,رئیس جمهور پس از افزایش شمار ابتلا و مرگ و میر...
6134,با پیروزی انقلاب اغلب هنرها از جمله موسیقی رشد...,، سعید تشکری: ادبیات انقلاب اسلامی و سقوط استب...,سعید تشکری: ادبیات انقلاب اسلامی و سقوط استبدا...
6135,مسعود فراستی بار دیگر در برنامه هفت به انتقاد ...,به گزارش خبرنگار فرهنگی، مسعود فراستی در میز ن...,مسعود فراستی در میز نقد برنامه «هفت» درباره سر...


Great! now lets use Hazm normalization to make predictions normal (not really necessary)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/final_test_dataset_with_predictions.csv')

normalizer=Normalizer()

tqdm.pandas()

df['normal_prediction'] = df['predicted_abstract'].progress_apply(normalizer.normalize)


df.to_csv('/content/drive/MyDrive/predictions_summary.csv', index=False)


100%|██████████| 6137/6137 [00:05<00:00, 1127.17it/s]


In [ ]:
df

,normal_abstract,normal_body,predicted_abstract,normal_prediction
0,تجارت ایران و روسیه در ۹ ماه سال ۱۴۰۰ از سه و ...,به گزارش، سید روح اله لطیفی، سخنگوی گمرک در آس...,سخنگوی گمرک گفت: از تجارت سه میلیون و ۵۵۸ هزار...,سخنگوی گمرک گفت: از تجارت سه میلیون و ۵۵۸ هزار...
1,در پی اظهارات محسن هاشمی رئیس شورای شهر تهران ...,به گزارش خبرنگار اجتماعی؛ محسن هاشمی رئیس شورا...,رئیس شورای شهر تهران در دویست و هشتاد و نهمین ...,رئیس شورای شهر تهران در دویست و هشتاد و نهمین ...
2,یک کارشناس آینده‌پژوهی با اشاره به بحران کاهش ...,به گزارش خبرنگار اجتماعی باشگاه خبرنگاران پویا...,رئیس پژوهشگاه علوم تحقیقات و فناوری اطلاعات گف...,رئیس پژوهشگاه علوم تحقیقات و فناوری اطلاعات گف...
3,رئیس‌جمهور در مراسم تشییع باشکوه و پرشور پیکر ...,به گزارش حوزه دولت، آیت‌الله سید ابراهیم رئیسی...,رئیس جمهور از تقاطع خیابان فلسطین به جمع تشییع...,رئیس‌جمهور از تقاطع خیابان فلسطین به جمع تشییع...
4,گروه استان‌ها- اداره میراث فرهنگی و گردشگری شه...,صیاد جعفری فرد در گفت‌وگو با خبرنگار تسنیم در ...,گروه استان ها- رئیس اداره میراث فرهنگی و گردشگ...,گروه استان ها- رئیس اداره میراث فرهنگی و گردشگ...
...,...,...,...,...
6132,رئیس شورای حاکمیتی سودان عادی‌سازی روابط با رژ...,به گزارش گروه بین‌الملل، پائیز سال گذشته بود ک...,سودان با میانجیگری آمریکا توافقنامه عادی سازی ...,سودان با میانجیگری آمریکا توافقنامه عادی‌سازی ...
6133,در روزهای اخیر، رئیس‌جمهور از بخش خصوصی درخواس...,به گزارش خبرنگار بهداشت و درمان؛ پیش از ایام ن...,رئیس جمهور پس از افزایش شمار ابتلا و مرگ و میر...,رئیس‌جمهور پس از افزایش شمار ابتلا و مرگ و میر...
6134,با پیروزی انقلاب اغلب هنرها از جمله موسیقی رشد...,، سعید تشکری: ادبیات انقلاب اسلامی و سقوط استب...,سعید تشکری: ادبیات انقلاب اسلامی و سقوط استبدا...,سعید تشکری: ادبیات انقلاب اسلامی و سقوط استبدا...
6135,مسعود فراستی بار دیگر در برنامه هفت به انتقاد ...,به گزارش خبرنگار فرهنگی، مسعود فراستی در میز ن...,مسعود فراستی در میز نقد برنامه «هفت» درباره سر...,مسعود فراستی در میز نقد برنامه «هفت» درباره سر...


First Metric is Bleu{1,2,3,4}, for that we can use nltk.translate.bleu_score but we are using hazm tokenizer for Persian text

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/final_test_dataset_with_predictions.csv')
from hazm import word_tokenize

df['normal_abstract'] = df['normal_abstract'].apply(word_tokenize)
df['predicted_abstract'] = df['predicted_abstract'].apply(word_tokenize)
smoothie = SmoothingFunction().method4

df['bleu1'] = df.apply(lambda row: sentence_bleu([row['normal_abstract']], row['predicted_abstract'], weights=(1, 0, 0, 0), smoothing_function=smoothie), axis=1)
df['bleu2'] = df.apply(lambda row: sentence_bleu([row['normal_abstract']], row['predicted_abstract'], weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie), axis=1)
df['bleu3'] = df.apply(lambda row: sentence_bleu([row['normal_abstract']], row['predicted_abstract'], weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie), axis=1)
df['bleu4'] = df.apply(lambda row: sentence_bleu([row['normal_abstract']], row['predicted_abstract'], weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie), axis=1)

print('Average BLEU-1 score:', df['bleu1'].mean())
print('Average BLEU-2 score:', df['bleu2'].mean())
print('Average BLEU-3 score:', df['bleu3'].mean())
print('Average BLEU-4 score:', df['bleu4'].mean())

Average BLEU-1 score: 0.3295322741811985
Average BLEU-2 score: 0.22883823298491263
Average BLEU-3 score: 0.18159789381972424
Average BLEU-4 score: 0.14708211786285305


For ROUGE score, I have done two things,

1.   Using the default ROUGE library.

2.   Use custom ROUGE for Persian language by hooshvare lab



In [ ]:
!pip install rouge

In [ ]:
import pandas as pd
from rouge import Rouge

df = pd.read_csv('/content/drive/MyDrive/final_test_dataset_with_predictions.csv')

rouge = Rouge()

df['rouge'] = df.apply(lambda row: rouge.get_scores(row['predicted_abstract'], row['normal_abstract'], avg=True), axis=1)

rouge_scores = df['rouge'].apply(pd.Series)
print('Average ROUGE-1 score:', rouge_scores.apply(lambda col: col.apply(lambda row: row['r'])).mean())
print('Average ROUGE-2 score:', rouge_scores.apply(lambda col: col.apply(lambda row: row['r'])).mean())
print('Average ROUGE-L score:', rouge_scores.apply(lambda col: col.apply(lambda row: row['r'])).mean())


Average ROUGE-1 score: rouge-1    0.368522
rouge-2    0.192291
rouge-l    0.325636
dtype: float64
Average ROUGE-2 score: rouge-1    0.368522
rouge-2    0.192291
rouge-l    0.325636
dtype: float64
Average ROUGE-L score: rouge-1    0.368522
rouge-2    0.192291
rouge-l    0.325636
dtype: float64


Using hooshvare implemented metric (cloning their git)

In [ ]:
!git clone https://github.com/hooshvare/pn-summary
!cp -a /content/pn-summary/scripts/rouge/ /content/
!cp -a /content/pn-summary/scripts/rouge_score/ /content/

Cloning into 'pn-summary'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 245 (delta 32), reused 30 (delta 30), pack-reused 209
Receiving objects: 100% (245/245), 43.08 MiB | 19.10 MiB/s, done.
Resolving deltas: 100% (134/134), done.
Updating files: 100% (43/43), done.


In [ ]:
import pandas as pd
from datasets import load_metric
from prettytable import PrettyTable

df = pd.read_csv('/content/drive/MyDrive/final_test_dataset_with_predictions.csv')

hyp = df['predicted_abstract'].tolist()
ref = df['normal_abstract'].tolist()

rouge = load_metric('/content/rouge/')

scores = rouge.compute(
    predictions=hyp,
    references=ref,
    rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"],
    use_stemmer=False,
    lang='fa'
)

table = PrettyTable(title='Rouge Metric')
table.field_names = ["Score", "Type", "Precision", "Recall", "F-Measure"]
for i, score_name in enumerate(scores):
    score = dict(scores[score_name]._asdict())
    for score_type in score:
        values = score[score_type]
        table.add_row([f'{score_name}', f'{score_type}', f'{values.precision * 100:0.2f} ', f'{values.recall * 100:0.2f}', f'{values.fmeasure * 100:0.2f}'])

    if i < len(scores) - 1:
        table.add_row([''] * len(table.field_names))
        table.add_row(['***'] * len(table.field_names))
        table.add_row([''] * len(table.field_names))

print(table.get_string())


<ipython-input-11-f4fe2340909a>:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric('/content/rouge/')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:848: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at /content/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   40.72   | 39.51  |   39.07   |
|   rouge1  | mid  |   41.35   | 40.10  |   39.66   |
|   rouge1  | high |   41.96   | 40.69  |   40.24   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   24.04   | 23.32  |   23.07   |
|   rouge2  | mid  |   24.77   | 24.00  |   23.77   |
|   rouge2  | high |   25.40   | 24.66  |   24.39   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   35.41   | 34.30  |   33.94   |
|   rougeL  | mid  |   36.02

In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 642.3 kB/s eta 0:00:00


For Persian Bert-Score I used “HooshvareLab/bert-fa-base-uncased”.

Scoring: The score function from the bert_score library is used to calculate the Precision (P), Recall (R), and F1 Score (F1) for the predicted texts against the reference texts. The language is set to ‘fa’ (Farsi) and the number of layers is set to 12.

In [ ]:
import pandas as pd
from bert_score import score

df = pd.read_csv('/content/drive/MyDrive/final_test_dataset_with_predictions.csv')

cands = df['predicted_abstract'].tolist()
refs = df['normal_abstract'].tolist()

model_name = "HooshvareLab/bert-fa-base-uncased"

P, R, F1 = score(cands, refs, lang="fa", model_type=model_name, num_layers=12, verbose=True)

print(f'Precision: {P.mean()}, Recall: {R.mean()}, F1 Score: {F1.mean()}')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

calculating scores...
computing bert embedding.


  0%|          | 0/189 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/96 [00:00<?, ?it/s]

done in 44.00 seconds, 139.48 sentences/sec
Precision: 0.5841580629348755, Recall: 0.5913935899734497, F1 Score: 0.5857959985733032


# Modified prediction

for prediction previusly I had used 256 and 64 max length. but some of them were longer and so they were truncated. Now by setting them higher we are gonna evaluate

In [ ]:
from transformers import AutoModelForSeq2SeqLM, MT5Tokenizer
from datasets import DatasetDict

dataset_dict = DatasetDict.load_from_disk("/content/drive/MyDrive/my_datasets")

model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/my_finetuned_model")
tokenizer = MT5Tokenizer.from_pretrained("/content/drive/MyDrive/my_finetuned_tokenizer")

train_dataset = dataset_dict['train']
val_dataset = dataset_dict['validation']
test_dataset = dataset_dict['test']


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
import pandas as pd
from tqdm import tqdm
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

df = pd.read_csv('/content/drive/MyDrive/NLP_HW4_test_dataset.csv')

predicted_abstracts = []

for text in tqdm(df['normal_body'], desc="Generating summaries"):
    inputs = tokenizer(text, truncation=True, max_length=512, padding='max_length', return_tensors="pt")

    inputs = inputs.to(device)

    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=256, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    predicted_abstracts.append(summary)

df['predicted_abstract'] = predicted_abstracts

df.to_csv('/content/drive/MyDrive/new_experiment.csv', index=False)


Generating summaries: 100%|██████████| 6137/6137 [1:53:18<00:00,  1.11s/it]


In [ ]:
df

,normal_abstract,normal_body,predicted_abstract
0,تجارت ایران و روسیه در ۹ ماه سال ۱۴۰۰ از سه و ...,به گزارش، سید روح اله لطیفی، سخنگوی گمرک در آس...,سخنگوی گمرک گفت: در ۹ ماهه امسال نسبت به ۹ ماه...
1,در پی اظهارات محسن هاشمی رئیس شورای شهر تهران ...,به گزارش خبرنگار اجتماعی؛ محسن هاشمی رئیس شورا...,رئیس شورای شهر تهران نسبت به اظهارات ریاست شور...
2,یک کارشناس آینده‌پژوهی با اشاره به بحران کاهش ...,به گزارش خبرنگار اجتماعی باشگاه خبرنگاران پویا...,یک آینده پژوهشی با اشاره به کم فرزندی خانواده ...
3,رئیس‌جمهور در مراسم تشییع باشکوه و پرشور پیکر ...,به گزارش حوزه دولت، آیت‌الله سید ابراهیم رئیسی...,رئیس جمهور از تقاطع خیابان فلسطین به جمع تشییع...
4,گروه استان‌ها- اداره میراث فرهنگی و گردشگری شه...,صیاد جعفری فرد در گفت‌وگو با خبرنگار تسنیم در ...,گروه استان ها - رئیس اداره میراث فرهنگی و گردش...
...,...,...,...
6132,رئیس شورای حاکمیتی سودان عادی‌سازی روابط با رژ...,به گزارش گروه بین‌الملل، پائیز سال گذشته بود ک...,سودان با میانجیگری آمریکا توافقنامه عادی سازی ...
6133,در روزهای اخیر، رئیس‌جمهور از بخش خصوصی درخواس...,به گزارش خبرنگار بهداشت و درمان؛ پیش از ایام ن...,رئیس جمهور پس از افزایش شمار ابتلا و مرگ و میر...
6134,با پیروزی انقلاب اغلب هنرها از جمله موسیقی رشد...,، سعید تشکری: ادبیات انقلاب اسلامی و سقوط استب...,در جریان انقلاب اسلامی، هنرها شانه به شانه یکد...
6135,مسعود فراستی بار دیگر در برنامه هفت به انتقاد ...,به گزارش خبرنگار فرهنگی، مسعود فراستی در میز ن...,مسعود فراستی در میز نقد برنامه «هفت» درباره سر...


BLEU scores

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/new_experiment.csv')
from hazm import word_tokenize

df['normal_abstract'] = df['normal_abstract'].apply(word_tokenize)
df['predicted_abstract'] = df['predicted_abstract'].apply(word_tokenize)
smoothie = SmoothingFunction().method4

df['bleu1'] = df.apply(lambda row: sentence_bleu([row['normal_abstract']], row['predicted_abstract'], weights=(1, 0, 0, 0), smoothing_function=smoothie), axis=1)
df['bleu2'] = df.apply(lambda row: sentence_bleu([row['normal_abstract']], row['predicted_abstract'], weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie), axis=1)
df['bleu3'] = df.apply(lambda row: sentence_bleu([row['normal_abstract']], row['predicted_abstract'], weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie), axis=1)
df['bleu4'] = df.apply(lambda row: sentence_bleu([row['normal_abstract']], row['predicted_abstract'], weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie), axis=1)

# Print average BLEU scores
print('Average BLEU-1 score:', df['bleu1'].mean())
print('Average BLEU-2 score:', df['bleu2'].mean())
print('Average BLEU-3 score:', df['bleu3'].mean())
print('Average BLEU-4 score:', df['bleu4'].mean())

Average BLEU-1 score: 0.33410592089570873
Average BLEU-2 score: 0.23126475172999142
Average BLEU-3 score: 0.1831961536661239
Average BLEU-4 score: 0.14801674672296813


It has improved a bit but very little

In [ ]:
import pandas as pd
from rouge import Rouge

df = pd.read_csv('/content/drive/MyDrive/new_experiment.csv')

rouge = Rouge()

df['rouge'] = df.apply(lambda row: rouge.get_scores(row['predicted_abstract'], row['normal_abstract'], avg=True), axis=1)

rouge_scores = df['rouge'].apply(pd.Series)
print('Average ROUGE-1 score:', rouge_scores.apply(lambda col: col.apply(lambda row: row['r'])).mean())
print('Average ROUGE-2 score:', rouge_scores.apply(lambda col: col.apply(lambda row: row['r'])).mean())
print('Average ROUGE-L score:', rouge_scores.apply(lambda col: col.apply(lambda row: row['r'])).mean())


Average ROUGE-1 score: rouge-1    0.378236
rouge-2    0.197865
rouge-l    0.334025
dtype: float64
Average ROUGE-2 score: rouge-1    0.378236
rouge-2    0.197865
rouge-l    0.334025
dtype: float64
Average ROUGE-L score: rouge-1    0.378236
rouge-2    0.197865
rouge-l    0.334025
dtype: float64


In [ ]:
!git clone https://github.com/hooshvare/pn-summary
!cp -a /content/pn-summary/scripts/rouge/ /content/
!cp -a /content/pn-summary/scripts/rouge_score/ /content/
import pandas as pd
from datasets import load_metric
from prettytable import PrettyTable

df = pd.read_csv('/content/drive/MyDrive/new_experiment.csv')

hyp = df['predicted_abstract'].tolist()
ref = df['normal_abstract'].tolist()

rouge = load_metric('/content/rouge/')

scores = rouge.compute(
    predictions=hyp,
    references=ref,
    rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"],
    use_stemmer=False,
    lang='fa'
)

table = PrettyTable(title='Rouge Metric')
table.field_names = ["Score", "Type", "Precision", "Recall", "F-Measure"]
for i, score_name in enumerate(scores):
    score = dict(scores[score_name]._asdict())
    for score_type in score:
        values = score[score_type]
        table.add_row([f'{score_name}', f'{score_type}', f'{values.precision * 100:0.2f} ', f'{values.recall * 100:0.2f}', f'{values.fmeasure * 100:0.2f}'])

    if i < len(scores) - 1:
        table.add_row([''] * len(table.field_names))
        table.add_row(['***'] * len(table.field_names))
        table.add_row([''] * len(table.field_names))

print(table.get_string())


fatal: destination path 'pn-summary' already exists and is not an empty directory.


/usr/local/lib/python3.10/dist-packages/datasets/load.py:848: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at /content/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   40.20   | 40.61  |   39.30   |
|   rouge1  | mid  |   40.84   | 41.22  |   39.90   |
|   rouge1  | high |   41.46   | 41.81  |   40.46   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   23.76   | 24.03  |   23.27   |
|   rouge2  | mid  |   24.43   | 24.65  |   23.89   |
|   rouge2  | high |   25.10   | 25.31  |   24.53   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   34.90   | 35.25  |   34.11   |
|   rougeL  | mid  |   35.54

In [ ]:
import pandas as pd
from bert_score import score

df = pd.read_csv('/content/drive/MyDrive/new_experiment.csv')

cands = df['predicted_abstract'].tolist()
refs = df['normal_abstract'].tolist()

model_name = "HooshvareLab/bert-fa-base-uncased"

P, R, F1 = score(cands, refs, lang="fa", model_type=model_name, num_layers=12, verbose=True)

print(f'Precision: {P.mean()}, Recall: {R.mean()}, F1 Score: {F1.mean()}')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

calculating scores...
computing bert embedding.


  0%|          | 0/189 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/96 [00:00<?, ?it/s]

done in 56.32 seconds, 108.97 sentences/sec
Precision: 0.584496259689331, Recall: 0.5979945659637451, F1 Score: 0.5891674757003784


#Conclusion
We fine-tuned a model to summarize a large dataset and got good results, even with limited resources.

#Limitations
Our main limit was GPU power. With more resources, we could improve our results. Future work could involve using bigger models, advanced techniques, and better and longer training phase.